In [1]:
from algorithms.preprocessing import get_data, word_by_sent, wbys_to_word, word_to_idx, idx_by_sent
# text = get_data('data/EI_original copy.txt')
text = get_data()
wbys = word_by_sent(text)
wordlist = wbys_to_word(wbys)
wtoi = word_to_idx(wordlist)
ibys = idx_by_sent(wbys, wtoi)

In [2]:
from algorithms.textrank import count_window, textrank_keyword, textrank_graph,keywords_to_nodes 
counter = count_window(ibys, 5)
mainkeywords = textrank_keyword(ibys, wordlist)
g_words = textrank_keyword(ibys, wordlist, onlyWords=True)

In [3]:
import algorithms.visualization as vis
import networkx as nx

cnt_draw = vis.counter_draw(counter,g_words)
IG = vis.initialGraph(cnt_draw,g_words)
vis.communityGraph(IG)
subnodes = keywords_to_nodes(mainkeywords)
nx.set_node_attributes(IG, subnodes)

comms = nx.get_node_attributes(IG, 'comm')
weights = nx.get_node_attributes(IG, 'weight')

In [4]:
import pandas as pd

In [5]:
pdc = pd.Series(comms, name='comm')
pdc

bars         0
bar          0
saturated    0
fat          0
energy       1
            ..
watch        7
example      6
granola      6
sugar        6
low          6
Name: comm, Length: 96, dtype: int64

In [6]:
pdw = pd.Series(weights, name='weight')
pdw

bars         52
bar          29
saturated    26
fat          26
energy       19
             ..
watch         3
example       3
granola       3
sugar         3
low           3
Name: weight, Length: 96, dtype: int64

In [7]:
data = pd.concat([pdc, pdw], axis=1)
data

,comm,weight
bars,0,52
bar,0,29
saturated,0,26
fat,0,26
energy,1,19
...,...,...
watch,7,3
example,6,3
granola,6,3
sugar,6,3


In [8]:
data = data.sort_values(by=['comm','weight'], ascending=[True, False])
data

,comm,weight
bars,0,52
bar,0,29
saturated,0,26
fat,0,26
oil,0,10
...,...,...
pick,7,4
general,7,4
equal,7,3
afford,7,3


In [58]:

def makeCircle(newlist):
    if(len(newlist) < 7):
        outercircle = []
        for w in newlist:
            outercircle.append({"name": w, "size": weights[w]})
        return outercircle
    
    SG = IG.subgraph(newlist) # 기존 그래프가 있어야 간편
    vis.communityGraph(SG)
    newcomms = nx.get_node_attributes(SG, 'comm')
    
    newweights = {}
    for k, v in newcomms.items():
        newweights[k] = weights[k]
        
    pdc = pd.Series(newcomms, name='comm')
    pdw = pd.Series(newweights, name='weight')
    df = pd.concat([pdc, pdw], axis=1)
    df = df.sort_values(by=['comm','weight'], ascending=[True, False])
    
    outercircle = []
    idx = 0
    for c in range(len(df.groupby('comm').size())):
        newList = []
        # make new list by each commynities
        for j, w in enumerate(df.index[df['comm'] == c]):
            if (j == 0):
                innercircle = {"name": df.iloc[idx].name}
                idx += df.groupby('comm').size()[c]
            else:
                newList.append(w)
        innercircle["children"] = makeCircle(newList)
        outercircle.append(innercircle)
            
    return outercircle


outercircle = {"name": "my_cp_v2", "children": []}
idx = 0
for c in range(2): #len(df.groupby('comm').size())
    newList = []
    # make new list by each commynities
    for j, w in enumerate(data.index[data['comm'] == c]):
        if (j == 0):
            innercircle = {"name": data.iloc[idx].name}
            idx += data.groupby('comm').size()[c]
        else:
            newList.append(w)
    innercircle["children"] = makeCircle(newList)
    outercircle["children"].append(innercircle)
print(outercircle)

{'name': 'my_cp_v2', 'children': [{'name': 'bars', 'children': [{'name': 'different', 'children': [{'name': 'snacks', 'size': 7}, {'name': 'meals', 'size': 7}, {'name': 'boost', 'size': 7}, {'name': 'meal', 'size': 7}, {'name': 'limit', 'size': 7}]}, {'name': 'bar', 'children': [{'name': 'saturated', 'children': [{'name': 'fat', 'size': 26}]}, {'name': 'nutritional', 'children': [{'name': 'vitamins', 'size': 10}, {'name': 'dinner', 'size': 10}, {'name': 'replacement', 'size': 9}]}, {'name': 'fiber', 'children': [{'name': 'oil', 'size': 10}]}]}, {'name': 'better', 'children': [{'name': 'vary', 'size': 8}, {'name': 'invert', 'size': 8}, {'name': 'cane', 'size': 8}]}]}, {'name': 'energy', 'children': [{'name': 'word', 'children': [{'name': 'like', 'size': 4}]}, {'name': 'gaps', 'children': [{'name': 'fortified', 'size': 5}, {'name': 'closely', 'size': 4}]}, {'name': 'significantly', 'children': [{'name': 'ingredients', 'size': 8}, {'name': 'palates', 'size': 4}]}, {'name': 'wide', 'childr